In [ ]:
#import libraries
import os
import pandas as pd
import re
import csv
from io import StringIO
from pathlib import Path
import numpy as np
import glob

In [ ]:
#read file
fullpath = Path(input("Enter the file name : "))

In [ ]:
#get rid of \ 
fullpath = Path(str(fullpath).replace(r'\ ',' '))

In [ ]:
#load csv to dataframe
#col_list = ['Date', 'Desc', 'Debit', 'credit']
#df = pd.read_csv(fullpath, usecols = [0,1,2,3], names = col_list)
df = pd.read_csv(fullpath)

In [ ]:
#get rid of unnecessary rows
bad_items = ['Scroll for more items', 'Description', 'Subtotal','Shipping']
df = df.loc[-df['Product image'].isin(bad_items),:]

In [ ]:
#get rid of Product Image column
df = df.drop(columns = 'Product image')

In [ ]:
#Add file date
date_pat = re.compile('\d{4}-\d{2}-\d{2}')
file_date = date_pat.findall(fullpath.name)[0]
df['Date'] = file_date

In [ ]:
#get rid of Pesos
df.Price = df.Price.str.replace('₱','')

In [ ]:
#extract content from description
df['Content'] = df.Description.str.extract(r'(\d+k?g?)', expand = False).str.extract(r'(\d+)').fillna(1)

In [ ]:
#extract units from content
df['Units'] = df.Description.str.extract(r'(\d+k?g?)', expand = False)
df['Units'] = df.Units.str.replace(r'(\d+)','').fillna('pcs')
df['Units'] = df.Units.replace(r'^\s*$',np.NaN, regex = True).fillna('pcs')

In [ ]:
#create item column containing basic item identifier
#remove Fresh
df['Item'] = df.Description.str.replace('Fresh ','')

In [ ]:
#get rid spaces
df.Item = df.Item.str.replace(r'\".+\"','')

In [ ]:
#get rid spaces
pattern = r'(\(.+\))'
df.Item = df.Item.str.replace(pattern,'')

In [ ]:
#get rid units stuck with text
pattern = r'(\d+k?g?)'
df.Item = df.Item.str.replace(pattern,'')

In [ ]:
#get rid of Fruits
pattern = 'Fruits '
df.Item = df.Item.str.replace(pattern,'')

In [ ]:
#change types
for col in ['Price', 'Content']:
    df[col] = df[col].astype('float64')

df.Date = df.Date.astype('datetime64')
df.Units = df.Units.astype('category')

In [ ]:
#lower case
df.Item = df.Item.str.lower()

In [ ]:
#export to csv
dest_path = (fullpath.parents[0] / ('Orders Freshmarket cleaned ' + file_date + '.csv'))
df.to_csv(dest_path, index = None, header=False)

In [ ]:
#food_categories = ['Pork','Fruits','Fish','Beef','Chicken']
#df.item.str.contains